# Tarea5: Series de Taylor

Fecha de envío del PR inicial: **viernes 21 de abril**

Fecha de aceptación del PR: **martes 25 de abril, antes de la clase**

---

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests_taylor.jl") para cada uno de los métodos que implementen.


In [1]:
"""
Definición de polinomios de Taylor, donde
...
"""
type Taylor{T<:Number}
    orden::Int64 #el orden de la serie de taylor.
    coff::Array{T,1}   #coeficientes en la serie de taylor.
end

Taylor

In [2]:
function Taylor(orden,coff)
    return Taylor(promote(orden,coff)...)    #al igual que la tarea3, se bhace la especificación de cómo estará compuesto la función taylor
end

Taylor{T<:Number}

In [3]:
a=[2,3,4,5,1.2]#es necesario recordar que sólo reconoce vectores con aunque sea un valor de punto flotante

5-element Array{Float64,1}:
 2.0
 3.0
 4.0
 5.0
 1.2

In [4]:
a[4]

5.0

In [5]:
Taylor(2,a)

Taylor{Float64}(2,[2.0,3.0,4.0,5.0,1.2])

Esto sólo es un ejemplo para ver que el nuevo type *Taylor* funciona correctamente

In [6]:
function xTaylor(n)
    w=zeros(n) #genera un vector de n ceros
    w=deleteat!(w,n) #el último elemento no es cero
    w=append!(w,1.0) #el último elemento del vector de orden cero es 1.0
    return Taylor(n,w)
end

xTaylor (generic function with 1 method)

In [7]:
xTaylor(2)

Taylor{Float64}(2,[0.0,1.0])

In [8]:
m=[1,2]

2-element Array{Int64,1}:
 1
 2

In [9]:
Taylor(2,m)

Taylor{Int64}(2,[1,2])

In [10]:
n=ans

Taylor{Int64}(2,[1,2])

In [11]:
n.coff

2-element Array{Int64,1}:
 1
 2

In [12]:
q=Taylor(2,[1,5])

Taylor{Int64}(2,[1,5])

In [13]:
n.coff+q.coff

2-element Array{Int64,1}:
 2
 7

In [14]:
size(n.coff)==size(q.coff)

true

In [15]:
import Base: +, -, *, /, ==

# Aqui se implementan los métodos necesarios para cada función
+(A::Taylor,B::Taylor)=Taylor(A.orden,A.coff+B.coff)
-(A::Taylor,B::Taylor)=Taylor(A.orden,A.coff-B.coff)

*(A::Real,B::Taylor)=Taylor(B.orden,A*B.coff)
*(A::Taylor,B::Real)=Taylor(A.orden,B*A.coff)

/(A::Taylor,B::Real)=Taylor(A.orden,(A.coff)/B)

==(A::Taylor,B::Taylor)=(A.orden==B.orden && A.coff==B.coff)

== (generic function with 119 methods)

In [51]:
# Muestren que su código funciona con tests adecuados

using Base.Test
include("runtest_taylor.jl")

LoadError: could not open file C:\KIRA\2017-2_TSFisComputacional-1\tareas\Carlos\runtest_taylor.jl

---

## Ejercicio 2

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas necesarias en `runtest_taylor.jl`.